# Anime Ratings Data 

### Source: https://www.kaggle.com/CooperUnion/anime-recommendations-database
We didn't really get the databowl data/ wanted to start with something I wanted to try for some time now. <br>
I wanted to predict whether a current figurine(say American comic kind/jpop culture kind,... basically merchandise will skyrocket in price or not) <br>
I wanted to find data on different collectables instead but such a dataset does not exist/ probably have to scrape or use an outdated API(?)
We want to predict ratings for anime

## Exploring time 


Importing stuff oop

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import joblib
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,forest 
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import metrics

C:\Users\tiny2\anaconda3\envs\fastai\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


###### *To note*
I made a folder in my data folder, databowl, i was lazy to change it <br>



In [ ]:
PATH = "databowl" 

In [ ]:
!ls {PATH}

prediction variable: rating. Can drop anime ID. <br>
to determine proportion to split for  test/training 

In [ ]:
df_raw = pd.read_csv(f'{PATH}/datasets_571_1094_anime.csv', low_memory=False)

### Saving for another time 
Well the file is kind of big and i want to sleep <br>
i will save my pickle format file in databowl folder so github wont read it :D 

In [3]:
import pickle

In [4]:
import os
print(os.getcwd())

D:\New folder\OneDrive - National University of Singapore\Documents\Stat_Soc_ML\dataBowl


In [ ]:
with open("databowl\\peepee", 'wb') as f:
    pickle.dump(df_raw,f)

### Note - the Edits for the dataframe before more pipelines
you can run this code instead of reading in the csv - think i included the tmp file 


In [89]:
df_raw = pickle.load(open("databowl\\peepee", 'rb')).drop('anime_id',axis=1) 
df_raw.episodes = pd.to_numeric(df_raw.episodes, errors = 'coerce')

In [90]:
df_raw = df_raw.dropna(subset=['rating','type'])
y = df_raw['rating']

In [91]:
df_raw['rating'].isna().sum() # rows with no ratings have been dropped 

0

Lookin good 
### Just append the genre onehotEncoded columns. 
But **DONT LOOK** at the final dataframe hahahah its so many NaNs.

In [74]:
# Put this code on hold first 
df_raw= df_raw.add(df_raw['genre'].str.get_dummies(sep=','),axis = 1)  #total have 82 genres

In [88]:
subset_raw = df_raw.sample(n=2000)
subset_raw.iloc[:,84:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 3946 to 7351
Data columns (total 4 columns):
members    0 non-null float64
name       0 non-null object
rating     0 non-null float64
type       0 non-null object
dtypes: float64(2), object(2)
memory usage: 78.1+ KB


### Doing splitting

do we need to use 67% for training set , then 33% test set? A: I just use fastai one
 https://machinelearningmastery.com/how-to-prepare-categorical-data-for-deep-learning-in-python/

Name looks too tedious to clean so just drop it

In [92]:
X_train, X_test, y_train,y_test= train_test_split(df_raw.drop(['rating','name'], axis = 1),y, test_size=0.20, random_state = 0)

In [28]:
X_train.tail()

,genre,type,episodes,members
4859,"Romance, School, Shoujo",Special,10.0,17684
3264,"Action, Adventure, Comedy, Fantasy, Shounen, S...",Special,1.0,4166
9847,"Comedy, Kids, Sci-Fi",TV,26.0,195
10801,"Comedy, Music, Slice of Life",Special,NaN,36
2732,"Comedy, Sci-Fi, Shounen, Space",Special,1.0,3626


In [29]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9651 entries, 520 to 2732
Data columns (total 4 columns):
genre       9611 non-null object
type        9651 non-null object
episodes    9498 non-null float64
members     9651 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 377.0+ KB


## Data cleaning/ dplyr time  + Model Fitting time
Using RandomForestRegressor 

https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
### We will pass these steps into our Pipeline - 
will have to make Pipeline another time 

### Step 1: SimpleImputer on the **type** column
### Step 2: LabelEncode categorical :type, name,genre
### Step 3: OneHotEncode Type column(not necessary now since we alr LabelEncode)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, LabelEncoder

The **transformers(focus)** and predictors **RandomForestRegressor**

In [93]:
numerical_features = ['episodes','members']
num_transformer = Pipeline([
    ('num_imputer', SimpleImputer(strategy='constant', fill_value=30))
])

types_transformer= Pipeline([
    ('one_hot', OneHotEncoder())
])




preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numerical_features),
        ('type', types_transformer, ['type'])]
)

randomForest = Pipeline(steps= [('preprocessor', preprocessor),
                                
                               ('regressor', RandomForestRegressor(n_jobs=-1, random_state = 0))])

In [94]:
from sklearn import set_config
set_config(display='diagram')
randomForest

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(fill_value=30,
                                                                                 strategy='constant'))]),
                                                  ['episodes', 'members']),
                                                 ('type',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder())]),
                                                  ['type'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=0))])

## PS: LabelEncoder is not made for features, so I will be switching it back to OneHot 
https://scikit-learn.org/stable/modules/preprocessing_targets.html#preprocessing-targets

In [95]:
randomForest.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(fill_value=30,
                                                                                 strategy='constant'))]),
                                                  ['episodes', 'members']),
                                                 ('type',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder())]),
                                                  ['type'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=0))])

### Shet. looks bad 

In [96]:
randomForest.score(X_test,y_test) 

0.37199312132805296

## put the type into numerical values, Movie, Music, OVA, ONA, TV, Special, type, and Blanks (total 7+1) 
i try to do with 'type' first instead of 'genre'.
#### will need to treat the missing values before applying label encoder
##### https://www.youtube.com/watch?v=ZS0hzcA5w9I


### Append this oneHotEncoded DF to the dataframe in Pipeline 

Strange: anime_id 9969 has some weird symbols in the name 

In [ ]:
alltypes=LabelEncoder()

In [ ]:
alltypes.fit_transform(df_raw.iloc[:,3]) 

In [ ]:
ct=ColumnTransformer(transformers=[('encode',OneHotEncoder(),[3])],remainder='passthrough')

In [ ]:
hi=pd.DataFrame(ct.fit_transform(df_raw))

In [ ]:
hi

### one hot encoding for genres(more than 1 genre to split) 

In [ ]:
abc=pd.get_dummies(df_raw.genre)

In [ ]:
## want to make separate columns from string #https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/

In [ ]:
yless=subset_rawless['rating']

In [ ]:
ygenre=subset_rawless['genre']

In [ ]:
ygenre

In [ ]:
def Convert(string):
    li = list(string.split(","))
    return li


In [ ]:
integer_encoded=label_encoder.fit_transform(values)


In [ ]:
onehot_encoder=OneHotEncoder(sparse=False)

In [ ]:
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)


In [ ]:
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


In [ ]:
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])


From data - genre can be splited and converted to one hot encoding(but do with piping ??) <br>
splitting into franchise name? probably I can find another dataset with the franchise and name and do a join. <br>


In [ ]:
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

In [ ]:
set_rf_samples(5000)

In [ ]:
import dill

## alternative Code to save session 
else i just use pickle

In [ ]:
dill.dump_session('notebook_sessions.db')

#### To load

In [ ]:
dill.load_session('notebook_session.db')

## Graph plotting time(Visualisation)

In [ ]:
from ggplot import *

In [ ]:
??ggplot

Collinearity between rating and members - no observable relationship(?)

In [ ]:
ggplot(subset_df, aes(x = 'rating', y = 'members')) +\
geom_point(alpha = 0.5) +\
ylim(0,500) 

From the plot - 
1. None(world) has no events - maybe its like the start page or something
2. possible linear relationship between event count and game time? 
3. World seems to be independent of game_time and event_count? 

In [ ]:
import json

In [ ]:
subset_df.event_data.iloc[2]

### Converting the json format of event_data into more rows:
1. media_type
2. total_duration(?)